<a href="https://colab.research.google.com/github/sydney-machine-learning/sentimentanalysis-USelections/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob 
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# read the tweets
trump_df = pd.read_csv("/content/drive/MyDrive/US_elections/cleaned_sample_100_biden.csv")
biden_df = pd.read_csv("/content/drive/MyDrive/US_elections/cleaned_sample_100_biden.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
#Define classes for cleaning our data and analyzing sentiment/subjectivity

#Class to clean tweets and remove text which is not useful for analysis
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

#Class to analyze the sentiment of tweets, either positive, neutral or negative
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity ==0:
        return 'Neutral'
    else:
        return 'Negative'

#Class to analyze the subjectivity of tweets, either subjective, neutral or not subjective
def analyze_subjectivity(tweet):
    analysis = TextBlob(tweet)
        
    if analysis.sentiment.subjectivity > 0:
        return 1
    elif analysis.sentiment.subjectivity == 0:
        return 0
    else:
        return -1 

#Class to analyze sentiment on a scale of -1 (negative) to 1 (positive)
def analyze_sentiment_d(tweet):
    analysis = TextBlob(clean_tweet(tweet))
        
    return analysis.sentiment.polarity

#Class to analyze subjectivity on a scale of -1 (not subjective) to 1 (subjective)
def analyze_subjectivity_d(tweet):
    analysis = TextBlob(clean_tweet(tweet))
        
    return analysis.sentiment.subjectivity

In [41]:
trump_df.head()

,user_id,tweet,long,lat,state_code,sentiment,text
0,531.0,TwitchSupport I know you are busy trying to st...,-73.992602,40.7422,NY,0.0,TwitchSupport I know you are busy trying to st...
1,2654.0,RickSantorum you are soo delusional that Trum...,-72.03236600000002,42.3789,MA,0.0,RickSantorum you are soo delusional that Trum...
2,2654.0,Wonderful irony that a major factor in AZ was ...,-72.03236600000002,42.3789,MA,0.0,Wonderful irony that a major factor in AZ was ...
3,2654.0,"OK, Trump Cabinet youve totally wimped out on ...",-72.03236600000002,42.3789,MA,0.0,"OK, Trump Cabinet youve totally wimped out on ..."
4,2654.0,Kinda think it would be cool to dust off an ol...,-72.03236600000002,42.3789,MA,0.0,Kinda think it would be cool to dust off an ol...


In [43]:
#Clean tweets
import re
trump_df['tweet'] = trump_df['tweet'].astype(str)
trump_df['clean_tweet'] = trump_df['tweet'].apply(lambda x: clean_tweet(x))

In [44]:
biden_df['tweet'] = biden_df['tweet'].astype(str)
biden_df['clean_tweet'] = biden_df['tweet'].apply(lambda x: clean_tweet(x))

In [45]:
trump_df.head()

,user_id,tweet,long,lat,state_code,sentiment,text,clean_tweet
0,531.0,TwitchSupport I know you are busy trying to st...,-73.992602,40.7422,NY,0.0,TwitchSupport I know you are busy trying to st...,TwitchSupport I know you are busy trying to st...
1,2654.0,RickSantorum you are soo delusional that Trum...,-72.03236600000002,42.3789,MA,0.0,RickSantorum you are soo delusional that Trum...,RickSantorum you are soo delusional that Trump...
2,2654.0,Wonderful irony that a major factor in AZ was ...,-72.03236600000002,42.3789,MA,0.0,Wonderful irony that a major factor in AZ was ...,Wonderful irony that a major factor in AZ was ...
3,2654.0,"OK, Trump Cabinet youve totally wimped out on ...",-72.03236600000002,42.3789,MA,0.0,"OK, Trump Cabinet youve totally wimped out on ...",OK Trump Cabinet youve totally wimped out on c...
4,2654.0,Kinda think it would be cool to dust off an ol...,-72.03236600000002,42.3789,MA,0.0,Kinda think it would be cool to dust off an ol...,Kinda think it would be cool to dust off an ol...
